<a href="https://colab.research.google.com/github/NagarjunaD024/Datascience-LLMS/blob/main/src/Frameworks%20and%20agents/langchain_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install langchain-openai
!pip install langchainhub
!pip install google-search-results
!pip install langchain openai sqlalchemy
!pip install -U langchain-community

In [ ]:
from langchain import hub
prompt = hub.pull('hwchase17/react')
print(prompt.template)

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [ ]:
from langchain.agents.load_tools import load_tools
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits.sql.base import create_sql_agent
from langchain_openai import ChatOpenAI


In [ ]:
from google.colab import drive, userdata,  output
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
llm = ChatOpenAI(api_key = userdata.get('OPENAI_API_KEY'),temperature=0, model='gpt-4o')
db_path = '/content/drive/My Drive/Colab Notebooks/DataScience+GPT/Data/structured_data/games.db'
db = SQLDatabase.from_uri(f'sqlite:///{db_path}')
extra_tools = load_tools(
    ['serpapi'], serpapi_api_key= "4efe4e98e408d433c0d2a842e320df23743b16d813d5fce429b5665367c7193c", llm=llm)

agent = create_sql_agent(
    llm=llm, db=db, verbose=True,
    agent_type='openai-tools',
    extra_tools=extra_tools)
agent.invoke({'input': 'What was the most sold game in Europe, and how is it played?'})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


games
Invoking: `sql_db_schema` with `{'table_names': 'games'}`



CREATE TABLE games (
	rank INTEGER, 
	name TEXT, 
	platform TEXT, 
	year INTEGER, 
	genre TEXT, 
	publisher TEXT, 
	americasales NUMERIC, 
	eusales NUMERIC, 
	japansales NUMERIC, 
	othersales NUMERIC, 
	globalsales NUMERIC
)

/*
3 rows from games table:
rank	name	platform	year	genre	publisher	americasales	eusales	japansales	othersales	globalsales
1	Wii Sports	Wii	2006	Sports	Nintendo	41.4900000000	29.0200000000	3.7700000000	8.4600000000	82.7400000000
2	Super Mario Bros.	NES	1985	Platform	Nintendo	29.0800000000	3.5800000000	6.8100000000	0.7700000000	40.2400000000
3	Mario Kart Wii	Wii	2008	Racing	Nintendo	15.8500000000	12.8800000000	3.7900000000	3.3100000000	35.8200000000
*/
Invoking: `sql_db_query_checker` with `{'query': 'SELECT name, eusales FROM games ORDER BY eusales DESC LIMIT 1;'}`


```sql
SELECT name, eusales FROM games ORDER

{'input': 'What was the most sold game in Europe, and how is it played?',
 'output': 'The most sold game in Europe is "Wii Sports," with sales of 29.02 million units.\n\n"Wii Sports" is a sports video game developed and published by Nintendo for the Wii console. It consists of five sports simulations, designed to demonstrate the motion-sensing capabilities of the Wii Remote. The sports included are tennis, baseball, bowling, golf, and boxing. Players use the Wii Remote to mimic actions performed in real-life sports, such as swinging a tennis racket or rolling a bowling ball. The game is known for its intuitive controls and accessibility, making it popular among a wide range of players.'}